In [ ]:
import numpy as np
import tensorflow as tf
import os

import sionna
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, ITURadioMaterial, PathSolver

import mitsuba as mi
import drjit as dr

%matplotlib
import matplotlib.pyplot as plt

# mi.set_variant('scalar_rgb')

In [ ]:
min = (0,0, 0)
max = (500, 500, 1000)

spacing = 25
num_tx = 10

scene_dir = "minis"
scene_files = [f for f in os.listdir(scene_dir) if f.endswith('.xml')]
print(scene_files)

In [ ]:
scene_file = scene_files[1]
scene = load_scene(os.path.join(scene_dir, scene_file))

try:
    os.mkdir(os.path.splitext(scene_file)[0])
except OSError:
    print('dir already exists')

In [ ]:
rx_points = []

x_start, y_start, _ = min
x_end, y_end, _ = max
xx = np.linspace(x_start, x_end, spacing)
yy = np.linspace(y_start, y_end, spacing)
ray_pos = np.array([(x, y, 1000) for x in xx for y in yy])

z_rays = mi.Ray3f(o=ray_pos.T, d=(0, 0, -1))
z_hits = scene.mi_scene.ray_intersect(z_rays)

for z_hit, valid in zip(z_hits.p.numpy().T, z_hits.is_valid()):
    if (not valid):
        continue
    # start = int(np.ceil(z_hit[2] / spacing) * spacing)
    start = int(z_hit[2] + 10)
    
    # print(start)
    for i in range(start, 200, spacing):
        # print([z_hit[0], z_hit[1], i])
        rx_points.append(np.array([z_hit[0], z_hit[1], i]))
    # rx_points.append(z_hit)
# print(rx_points)

In [ ]:
## Include points that did not have a valid hitpoint due to lack of floor right now
for i, v in enumerate(z_hits.is_valid()):
    if(not v):
        r = ray_pos[i]
        for i in range(0, 200, spacing):
            rx_points.append(np.array([r[0], r[1], i]))
        # rx_points.append(np.array([r[0], r[1], 0]))

In [ ]:
scene.frequency # Defaults to 3.5GHz

mat = ITURadioMaterial(name='concrete', itu_type='concrete', thickness=0.1, scattering_coefficient=0.0, xpd_coefficient=0.0)

# Set all objects to concrete for now
for obj in scene.objects.values():
    obj.radio_material = mat

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=1,
                          horizontal_spacing=1,
                          polarization="V",
                          pattern="iso")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

# Create transmitter
# tx = Transmitter(name="tx",
#               position=[200,200,100],
#               orientation=[0,0,0])
# scene.add(tx)

In [ ]:
# Create receivers
for rx_i in range(len(rx_points)):
#     print(rx_points[rx_i])
    rx = Receiver(name=f"rx_{rx_i}",
            position=rx_points[rx_i],
            orientation=[0,0,0])
    scene.add(rx)

In [ ]:
# tx_pos = []
# for tx_i in range(num_tx):
    
points = np.array(rx_points)
filtered = points[(points[:,2] <= 50)]
# print(filtered)
filtered[:,2] += 5
# print(filtered)

np.random.seed(42)
c = np.random.choice(filtered.shape[0], size=num_tx, replace=False)
print(c)

tx_pos = filtered[c]

for tx_i in range(num_tx):
    tx = Transmitter(name=f"tx_{tx_i}",
            position=tx_pos[tx_i],
            orientation=[0,0,0])
    scene.add(tx)

In [ ]:
p_solver  = PathSolver()
paths = p_solver(scene=scene,
                 max_depth=5,
                 los=False,
                 specular_reflection=True,
                 diffuse_reflection=False,
                 refraction=False,
                 synthetic_array=False,
                 seed=41)

In [ ]:
# import cProfile
# p_solver  = PathSolver()

# cProfile.run(p_solver(scene=scene,
#                  max_depth=5,
#                  los=False,
#                  specular_reflection=True,
#                  diffuse_reflection=False,
#                  refraction=False,
#                  synthetic_array=False,
#                  seed=41))


In [ ]:
scene.preview(paths=paths)

Export fields to csv's

transmitter locs
receiver locs
a
primitives
pickle paths obj


In [ ]:
# scene.transmitters.
paths.sources
# paths.a
# paths.valid.shape
